In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE126132"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE126132"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE126132.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE126132.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE126132.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Distinct fibroblast functional states drive clinical outcomes in ovarian cancer and are regulated by TCF21"
!Series_summary	"Recent studies indicate that cancer-associated fibroblasts (CAFs) are phenotypically and functionally heterogeneous. However, little is known about CAF subtypes and the roles they play in cancer progression. Here we identify and characterize two CAF subtypes that coexist within high grade serous ovarian cancers: Fibroblast activation protein (FAP)-high (FH) CAFs resemble the classical myofibroblast-type CAF, whereas FAP-low (FL) CAFs possesses a preadipocyte-like molecular signature."
!Series_overall_design	"High-grade serous ovarian cancer single cell suspensions of 12 patients were stained for fluorescence activated cell sorting (FACS)  CD31-CD45-EpCAM-CD49e+ (CAFs) cells,  and CD31-CD45-EpCAM+CD133+ (epithelial) cells were gated for sorting based on FMO controls. Cells were sorted and  RNA was extracted immediately using 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset and extract clinical features

# 1. Gene Expression Data Availability
is_gene_available = True  # Dataset appears to contain gene expression data (not just miRNA or methylation)

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Ovarian Cancer):
# From the cell type field (key 3), we can identify cancer-associated fibroblasts (CAFs) vs epithelial cells
trait_row = 3  # 'cell type' field contains information about cell types

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert cell type information to binary trait (0 for non-CAF, 1 for CAF)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # CAF (cancer-associated fibroblast) is assigned 1, epithelial is assigned 0
    if 'CAF' in value:
        return 1
    elif 'epithelial' in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age information (not available in this dataset)"""
    return None  # Age data not available

def convert_gender(value):
    """Convert gender information (not available in this dataset)"""
    return None  # Gender data not available

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait data is available)
if trait_row is not None:
    # Sample characteristics dictionary from the output
    sample_chars = {
        0: ['subject id: 65846', 'subject id: 67794', 'subject id: 68584', 'subject id: 70535', 'subject id: 70924', 'subject id: 71029', 'subject id: 71377', 'subject id: 71423', 'subject id: 71853', 'subject id: 72130', 'subject id: 72143', 'subject id: 72199'],
        1: ['tissue: high-grade serous ovarian cancer (HGSOC)'] * 12,  # Repeat for all samples
        2: ['cell marker: CD31-CD45-EpCAM-CD49e+', 'cell marker: CD31-CD45-EpCAM+CD133+', 'cell marker: CD31-CD45-EpCAM+CD133+', 
            'cell marker: CD31-CD45-EpCAM-CD49e+', 'cell marker: CD31-CD45-EpCAM+CD133+', 'cell marker: CD31-CD45-EpCAM+CD133-',
            'cell marker: CD31-CD45-EpCAM-CD49e+', 'cell marker: CD31-CD45-EpCAM+CD133+', 'cell marker: CD31-CD45-EpCAM+CD133-',
            'cell marker: CD31-CD45-EpCAM-CD49e+', 'cell marker: CD31-CD45-EpCAM+CD133+', 'cell marker: CD31-CD45-EpCAM+CD133-'],
        3: ['cell type: CAF', 'cell type: epithelial', 'cell type: epithelial', 
            'cell type: CAF', 'cell type: epithelial', 'cell type: epithelial',
            'cell type: CAF', 'cell type: epithelial', 'cell type: epithelial',
            'cell type: CAF', 'cell type: epithelial', 'cell type: epithelial']
    }
    
    # Create a DataFrame with proper structure for geo_select_clinical_features
    # Rows are feature types, columns are samples
    sample_ids = [val.split(': ')[1] for val in sample_chars[0]]
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=sample_ids)
    
    # Fill the DataFrame with values
    for row_idx, values in sample_chars.items():
        for col_idx, value in enumerate(values):
            if col_idx < len(sample_ids):  # Ensure we don't go out of bounds
                clinical_data.loc[row_idx, sample_ids[col_idx]] = value
    
    # Use the library function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview_result = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview_result)
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the extracted clinical features to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'65846': [1.0], '67794': [0.0], '68584': [0.0], '70535': [1.0], '70924': [0.0], '71029': [0.0], '71377': [1.0], '71423': [0.0], '71853': [0.0], '72130': [1.0], '72143': [0.0], '72199': [0.0]}
Clinical features saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE126132.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (ILMN_*) are Illumina BeadArray probe IDs and not standard human gene symbols.
# They need to be mapped to official gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the output from previous steps, we need to map:
# - 'ID' column in gene_annotation to the index of gene_data (ILMN_* identifiers)
# - 'Symbol' column in gene_annotation to get the gene symbols

# 2. Using get_gene_mapping to extract the ID and Symbol columns from gene_annotation
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Shape of the gene expression data after mapping: {gene_data.shape}")
print("First 5 genes after mapping:")
print(gene_data.index[:5])


Gene mapping preview (first 5 rows):
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Shape of the gene expression data after mapping: (21464, 34)
First 5 genes after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")
print("First 5 genes after normalization:")
print(normalized_gene_data.index[:5])

# Save the normalized gene data to the specified file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data from the file
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data structure:")
print(clinical_data.head())
print("Clinical data columns:", clinical_data.columns.tolist())

# Print diagnostic information
print("Clinical data shape:", clinical_data.shape)
print("Normalized gene data shape:", normalized_gene_data.shape)

# Make sure clinical data is properly formatted before linking
# Rename the row to match the trait variable name
clinical_data = clinical_data.rename(index={"Ovarian_Cancer": trait})

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
print("Linked data shape:", linked_data.shape)
print("Linked data columns (first 10):", linked_data.columns[:10].tolist())
print("Linked data trait column null count:", linked_data[trait].isna().sum())

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether features are severely biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from CAF vs epithelial cells in ovarian cancer patients."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Shape of normalized gene data: (20259, 34)
First 5 genes after normalization:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE126132.csv
Clinical data structure:
                65846  67794  68584  70535  70924  71029  71377  71423  71853  \
Ovarian_Cancer    1.0    0.0    0.0    1.0    0.0    0.0    1.0    0.0    0.0   

                72130  72143  72199  
Ovarian_Cancer    1.0    0.0    0.0  
Clinical data columns: ['65846', '67794', '68584', '70535', '70924', '71029', '71377', '71423', '71853', '72130', '72143', '72199']
Clinical data shape: (1, 12)
Normalized gene data shape: (20259, 34)
Linked data shape: (46, 20260)
Linked data columns (first 10): ['Ovarian_Cancer', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1']
Linked data trait column null count: 34
Quartiles for 'Ovarian_Cancer':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Ovarian_Cancer' in this dataset is fine.

Abnormality detected in the cohort: GSE126132. Preprocessing failed.
Da